In [ ]:
"""
okitamark
OpenAI API Stream

LLMサーバを起動する
```
llamacpp-server  \
--chat-template vicuna \
--threads-batch 8 \
--threads-http 8 \
--model /home/users/model/vicuna-7b-v1.5.Q8_0.gguf \
--ctx-size 4095 \
--embeddings \
--parallel 8 \
--cont-batching \
--n-gpu-layers 33 \
--host 0.0.0.0 \
--port 8080
```
"""
## モデルとログファイルをセットする
MODEL_NAME = 'vicuna-7b-v1.5.Q8_0' ## API model名
BASE_URL = "http://localhost:8080/v1/" ## OpenAI API Endpoint

# LOG_FILE = __file__.replace('.py', '.log') ## デフォルトログファイル名, jupyterの時はNone
LOG_FILE = None #'test.log' ## ファイル名 or None


"""
Argument, Logging
"""
import argparse
import logging

## 引数の処理
## jupyterの場合コメント
# parser = argparse.ArgumentParser(description='OkitaMark')
# parser.add_argument('--model', type=str, default=MODEL_NAME)
# parser.add_argument('--log', type=str, default=LOG_FILE)
# args = parser.parse_args()
# MODEL_NAME = args.model
# LOG_FILE = args.log

## LOG_FILEが指定されていたら、ログファイルも出力する
if LOG_FILE is not None:
    handlers=[logging.FileHandler(LOG_FILE, mode='w'), logging.StreamHandler()]
else:
    handlers=[logging.StreamHandler()]

## Logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s', handlers=handlers) ## Message Only
LOG = logging.getLogger(__name__)


LOG.info('MODEL_NAME: ' + MODEL_NAME)
if LOG_FILE is not None: LOG.info('LOG_FILE:' + LOG_FILE)



"""
Base
"""
import os, time
import subprocess, argparse
## Warning非表示
import warnings
warnings.simplefilter('ignore')

## Logging
# import logging
# logging.basicConfig(level=logging.DEBUG, format='%(message)s') ## Message Only
# LOG = logging.getLogger(__name__)

## Util ##
## メモリ関連
## GPU Info
def gpu_info():
    gpucmd = 'nvidia-smi --query-gpu=name --format=csv,noheader'
    gpuinfo = subprocess.check_output(gpucmd, shell=True)
    return 'GPU device: ' + gpuinfo.decode()

def gpu_mem():
    gpucmd = 'nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used --format=csv,,noheader'
    gpuinfo = subprocess.check_output(gpucmd, shell=True)
    return 'GPU memory: ' + gpuinfo.decode()


## 推論
## LLMサーバ側でシステムプロンプトを指定していない場合、systemにシステムプロンプトを指定する
import openai
model_name = MODEL_NAME ## LLM Model Name: gpt-3.5-turbo, text-davinci-003

def predict(prompt, max_token=4095, temperature=0.1, top_p=0.1, title='', system=''):
    # token_ids = tokenizer.encode(prompt, return_tensors="pt")
    # n_token_input = len(token_ids[0])
    LOG.info('**'+title+'************************************************')
    LOG.info('**'+model_name)
    LOG.info('**'+gpu_info().rstrip('\n'))
    LOG.info('**'+gpu_mem().rstrip('\n'))
    LOG.info('**input:')
    LOG.info(prompt)
    ##　推論開始
    stime = time.perf_counter() ## 計測開始
    openai.api_key = "EMPTY" ## Dummy
    openai.base_url = BASE_URL ## OpenAI API Endpoint

    """ Chat completions """
    response = openai.chat.completions.create(
      model=model_name,
      max_tokens=max_token,
      temperature=temperature,
      top_p=top_p, 
      seed=0,
      stream=True, ## Streaming output
      messages=[
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
      ]
    )

    LOG.info('**output')
    ## Streaming output
    for chunk in response:
        c = chunk.choices[0].delta.content
        if c is not None:
            print(c, end='')
    tm = time.perf_counter() - stime ## 計測終了
    # output = response.choices[0].message.content
    # n_token_output = response.usage.completion_tokens
    # n_token_input = response.usage.prompt_tokens
    # n_token_total = response.usage.total_tokens

            
    ## 計測結果
    # LOG.info(output) ## Streamingではコメントアウト
    # LOG.info('**Result: %s, Time: %.2f, Input: %d, Output: %d, Total: %d, Token/sec: %.1f' % (title, tm, n_token_input, n_token_output, n_token_total, n_token_output/tm)) ## 終了時間, 出力文字数    
    LOG.info('**Result: %s, %s' % (title, gpu_mem().rstrip('\n')))
    LOG.info('\n\n')


## Prompt template
def qa_openai(input):
    return input
qa = qa_openai


predict(qa('自己紹介をしてください。'))